In [1]:
import findspark
findspark.init()

import pyspark

In [2]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

sc = SparkContext()
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/03 21:20:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.csv("flights_small.csv", header=True,
                      inferSchema=True)

In [4]:
num_rows = data.count()
num_columns = len(data.columns)
print(f"Shape: ({num_rows}, {num_columns})")

Shape: (10000, 16)


In [5]:
data.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)



In [6]:
print("Number of rows: ", data.count())

Number of rows:  10000


In [7]:
data.createOrReplaceTempView("flights_small")

#create the Dataframe from view
flights = spark.table("flights_small")

#show the head
flights.show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 3 rows



In [8]:
#add more a column of duration_hrs
flights = flights.withColumn("duration_hrs", flights.air_time/60)

In [9]:
flights.show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|         2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|        1.85|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 3 rows



In [10]:
# show the first 3 observations
for row in flights.head(3):
    print(row)
    print('\n')

Row(year=2014, month=12, day=8, dep_time='658', dep_delay='-7', arr_time='935', arr_delay='-5', carrier='VX', tailnum='N846VA', flight=1780, origin='SEA', dest='LAX', air_time='132', distance=954, hour='6', minute='58', duration_hrs=2.2)


Row(year=2014, month=1, day=22, dep_time='1040', dep_delay='5', arr_time='1505', arr_delay='5', carrier='AS', tailnum='N559AS', flight=851, origin='SEA', dest='HNL', air_time='360', distance=2677, hour='10', minute='40', duration_hrs=6.0)


Row(year=2014, month=3, day=9, dep_time='1443', dep_delay='-2', arr_time='1652', arr_delay='2', carrier='VX', tailnum='N847VA', flight=755, origin='SEA', dest='SFO', air_time='111', distance=679, hour='14', minute='43', duration_hrs=1.85)




In [11]:
# filter flights by passing a string
long_flights1 = flights.filter("distance > 2000")
print("Number of rows (distance > 2000): ", long_flights1.count())

Number of rows (distance > 2000):  1481


In [12]:
long_flights1.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|     duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|              6.0|
|2014|    1| 13|    2156|       -9|     607|      -15|     AS| N597AS|    24|   SEA| BOS|     290|    2496|  21|    56|4.833333333333333|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
only showing top 2 rows



In [13]:
# construct the "query"
query = '''SELECT * FROM flights_small WHERE distance > 2000'''
# apply the SQL "query"
long_flights1_sql = spark.sql(query)
long_flights1_sql.count()

In [ ]:
time_flights = flights.filter("air_time >= 300 and air_time <= 600")
time_flights.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         6.0|
|2014|   12|  4|     954|       -6|    1348|      -17|     HA| N395HA|    29|   SEA| OGG|     333|    2640|   9|    54|        5.55|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 2 rows



In [ ]:
time_flights.count()

440

In [ ]:
time_flights_sql = spark.sql('''SELECT * FROM flights_small WHERE air_time between 300 and 600''')
time_flights_sql.count()

440

In [ ]:
# select set of columns
selected1 = flights.select(flights.origin, flights.dest, flights.carrier)

# define first filter
filterA = flights.origin == "SEA"

# define second filter
filterB = flights.dest == "PDX"

# filter the data, first by filterA then filterB
selected2 = selected1.filter(filterA).filter(filterB)
selected2.show(5)


+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
+------+----+-------+
only showing top 5 rows



In [ ]:
selected2.count()

157

In [ ]:
# define avg_speed
avg_speed = (flights.distance/(flights.air_time/60)).alias("avg_speed")

# select the correct columns
speed1 = flights.select("origin", "dest", "tailnum", avg_speed)
speed1.show(2)

+------+----+-------+-----------------+
|origin|dest|tailnum|        avg_speed|
+------+----+-------+-----------------+
|   SEA| LAX| N846VA|433.6363636363636|
|   SEA| HNL| N559AS|446.1666666666667|
+------+----+-------+-----------------+
only showing top 2 rows



In [ ]:
# create the same table using a SQL expression
speed2 = flights.selectExpr("origin", "dest", "tailnum",
                            "distance/(air_time/60) as avg_speed")
speed2.show(2)

+------+----+-------+-----------------+
|origin|dest|tailnum|        avg_speed|
+------+----+-------+-----------------+
|   SEA| LAX| N846VA|433.6363636363636|
|   SEA| HNL| N559AS|446.1666666666667|
+------+----+-------+-----------------+
only showing top 2 rows



In [ ]:
from pyspark.sql.types import IntegerType

In [ ]:
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))

In [ ]:
# find the shortest time from PDX in terms of air_time
flights.filter(flights.origin == "PDX").groupBy().min("air_time").show()

+-------------+
|min(air_time)|
+-------------+
|           24|
+-------------+



In [ ]:
# find the longest distance from SEA in terms of distance
flights.filter(flights.origin == "SEA").groupBy().max("distance").show()

+-------------+
|max(distance)|
+-------------+
|         2724|
+-------------+



In [ ]:
# total hours in the air
flights.withColumn("duration_hrs",
                   flights.air_time/60).groupBy().sum("duration_hrs").show()

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



In [ ]:
# group by tailnum
by_plane = flights.groupBy("tailnum")

# number of flights  each plane made
by_plane.count().show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



In [ ]:
# group by origin
by_origin = flights.groupBy("origin").count()
by_origin.show()

+------+-----+
|origin|count|
+------+-----+
|   SEA| 6754|
|   PDX| 3246|
+------+-----+



In [ ]:
from pyspark.sql.functions import avg

# average air_time
flights.groupBy("origin").avg("air_time").show()

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



In [ ]:
flights = flights.withColumn("dep_delay",
                             flights["dep_delay"].cast(IntegerType()))

In [ ]:
# group by month and dest
by_month_dest = flights.groupBy("month", "dest")

# average departure delay by month and destination
by_month_dest.avg("dep_delay").show()

AnalysisException: "dep_delay" is not a numeric column. Aggregation function can only be applied on a numeric column.

In [ ]:
import pyspark.sql.functions as F

In [ ]:
# standard devitation of departure delay
by_month_dest.agg(F.stddev("dep_delay")).show()

NameError: name 'by_month_dest' is not defined